Figure of different solution to the opaque HI model

In [ ]:

%matplotlib inline

from astropy.io import fits
from astropy.wcs import WCS
import astropy.units as u
import os
import numpy as np
import matplotlib.pyplot as plt
from spectral_cube import Projection
from scipy import ndimage as nd
from astropy.stats import histogram as astro_hist
from tqdm import tqdm
from corner import hist2d

osjoin = os.path.join

repo_path = os.path.expanduser("~/ownCloud/project_code/ThickHIFitting/")

figures_path_png = osjoin(repo_path, "figures/png")
figures_path_pdf = osjoin(repo_path, "figures/pdf")


def save_figure(fig, plot_name):
    fig.savefig(f"{figures_path_pdf}/{plot_name}.pdf")
    fig.savefig(f"{figures_path_png}/{plot_name}.png")


paths_script = os.path.join(repo_path, "paths.py")
exec(compile(open(paths_script, "rb").read(), paths_script, 'exec'))

plotstyle_script = os.path.join(repo_path, "plotting_styles.py")
exec(compile(open(plotstyle_script, "rb").read(), plotstyle_script, 'exec'))

model_script = os.path.join(repo_path, "gaussian_model.py")
exec(compile(open(model_script, "rb").read(), model_script, 'exec'))

thickHI_model_script = os.path.join(repo_path, "thickHI_model.py")
exec(compile(open(thickHI_model_script, "rb").read(),
             thickHI_model_script, 'exec'))
             

In [ ]:
save_figures = True

In [ ]:
def gaussian(vels, amp, vcent, sigma):
    return amp * np.exp(- 0.5 * ((vels - vcent) / sigma)**2)

In [ ]:
vels = np.arange(-40, 40.2, 0.42) * u.km / u.s
vels

In [ ]:
Tspin = 10**1.7 * u.K
# Tspin = 10**2 * u.K

c_s = np.sqrt(c.k_B * Tspin / (1.4 * c.m_p)).to(u.km / u.s)

sigma_nt = 4 * u.km / u.s
# sigma_nt = 12 * u.km / u.s
sigma = np.sqrt(sigma_nt**2 + c_s**2)

NHI = 1e22 * u.cm**-2

Tpeak = (5.49e-19 * NHI.value / (np.sqrt(2 * np.pi) * sigma.value)) * u.K

print(Tpeak)

# Tpeak = 200. * u.K

print(c_s)
print(sigma)

vcent = 0.0 * u.km / u.s


In [ ]:
mod_thickHI = isoturbHI_simple(vels.to(u.km / u.s),
                               Tspin,
                               sigma,
                               Tpeak,
                               vcent)

mod_thinHI = gaussian(vels, Tpeak, vcent, sigma)

mod_thickHI_tau = mod_thinHI / Tspin

In [ ]:
opaque_vels = vels[np.where(mod_thickHI_tau.value > 1)[0]]

print(opaque_vels[0], opaque_vels[-1])

In [ ]:
plt.plot(vels.value, mod_thickHI)
plt.plot(vels.value, mod_thinHI)

In [ ]:
onecolumn_figure()

plt.plot(vels.value, mod_thickHI)

plt.grid(True)

plt.ylabel("Brightness Temperature (K)")
plt.xlabel(r"Velocity (km s$^{-1}$")


In [ ]:
onecolumn_twopanel_figure()

fig, axs = plt.subplots(2, 1)

axs[0].plot(vels.value, mod_thickHI)

axs[0].set_ylabel("Brightness Temperature (K)")

axs[1].plot(vels.value, mod_thickHI_tau)

axs[1].set_ylabel(r"HI Optical Depth $\tau$")

axs[1].set_xlabel(r"Velocity (km s$^{-1}$")

axs[0].grid(True)
axs[1].grid(True)

axs[0].axvline(opaque_vels[0].value, alpha=0.5, linewidth=4, color='orange')
axs[0].axvline(opaque_vels[-1].value, alpha=0.5, linewidth=4, color='orange')
axs[1].axvline(opaque_vels[0].value, alpha=0.5, linewidth=4, color='orange')
axs[1].axvline(opaque_vels[-1].value, alpha=0.5, linewidth=4, color='orange')

axs[1].axhline(1., alpha=0.5, linewidth=4, color='orange')
# axs[1].text(-30, 1.4, r"$\tau=1$", color='orange', fontsize=15)
axs[1].text(-35, 1.4, r"$\tau=1$", color='k', fontsize=15)

fig.subplots_adjust(hspace=0.05)

if save_figures:
    save_figure(fig, "opaque_hi_model_example_withopacity")
